# INF325 2023 - Trabalho Banco de Dados

Neste projeto visamos elaborar uma modelagem e implementação de banco de dados que visa considerar a persistência de dados em uma plataforma de e-commerce (MarketPlace).
Mais especificamente iremos considerar a tratativa dada ao registro de reclamações recebidas, que devem disponibilizar ao usuário  a possibilidade de registrar e acompanhar as reclamações geradas no MarketPlace.

### Grupo 02
- Andressa Melo
- Ana Marina Araujo Santos
- Bruno Paulo
- Danilo Bastos Hernandes
- Gabriel Leite Lopes
- Juliana Lima 

### CQL Scripts para criar banco de dados e schema

-- abrir interface de consulta
! cqlsh

-- criando keyspace (Banco de Dados)
-- Necessário revisar parâmetros da query para justificar no relatório
CREATE KEYSPACE reclamacoes WITH REPLICATION = { 'class' : 'SimpleStrategy' , 'replication_factor' : 3 };

-- usando keyspace
USE  reclamacoes;

-- criando primeira tabela (Column family ou Table)
CREATE TABLE complaint (
ticket_id uuid,
user_id uuid,
order_id uuid,
product_id uuid,
seller_id uuid,
creation_date timestamp,
title text,
type text,
description text,
status text,
replies map<timestamp, text>,
PRIMARY KEY (ticket_id)
);

CREATE TABLE user (
id uuid,
cpf text,
name varchar,
address text,
country varchar,
email varchar,
PRIMARY KEY (id)
);
CREATE TABLE seller (
id uuid,
cnpj text,
name varchar,
address text,
country varchar,
email varchar,
PRIMARY KEY (id)
);


CREATE TABLE product (
id uuid,
name text,
description text,
price decimal,
images list<blob>,
sellers list<uuid>,
PRIMARY KEY (id)
);


CREATE TABLE order_info (
id uuid,
buyer_id uuid,
products_quantity map<uuid, int>,
sellers list<uuid>,
delivery_address text,
total_cost decimal,
PRIMARY KEY (id)
);

### Scripts Python

In [1]:
! pip install cassandra-driver

Defaulting to user installation because normal site-packages is not writeable


In [9]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'], port=9042)
    session = cluster.connect()

    session.execute("CREATE KEYSPACE reclamacoes WITH REPLICATION = { 'class' : 'SimpleStrategy' , 'replication_factor' : 3 }")
    session.set_keyspace("reclamacoes")

    query = "CREATE TABLE complaint (ticket_id uuid, user_id uuid, order_id uuid, product_id uuid, seller_id uuid,"
    query = query + "creation_date timestamp, title text, type text, description text, status text, replies map<timestamp, text>,"
    query = query + "PRIMARY KEY (ticket_id));"
    
    session.execute(query)

except Exception as e:
    print(e)

In [26]:
! python3 insertDataComplaint.py

In [27]:
from cassandra.cluster import Cluster
import uuid
from datetime import datetime, timedelta
import random
from collections import Counter

# Endereços dos nós do Cassandra (substitua pelos endereços reais do seu cluster)
enderecos_cassandra = ['127.0.0.1']

# Porta padrão do Cassandra
porta_cassandra = 9042

# Chaves do keyspace e nome da tabela (substitua pelos valores reais)
keyspace = 'reclamacoes'
nome_tabela = 'complaint'

# Função para estabelecer a conexão com o cluster do Cassandra
def conectar_cassandra():
    try:
        cluster = Cluster(enderecos_cassandra, port=porta_cassandra)
        session = cluster.connect(keyspace=keyspace)
        return cluster, session
    except Exception as e:
        print(f"Erro ao conectar ao Cassandra: {e}")
        return None, None

# Função para executar uma query no Cassandra
def executar_query(session, query):
    try:
        return session.execute(query)
    except Exception as e:
        print(f"Erro ao executar a query: {e}")

# Estabelecer a conexão
cluster, session = conectar_cassandra()
    

In [28]:
#Consulta: Produtos com mais reclamações
query = "SELECT product_id FROM complaint;"
rows = session.execute(query)
prod_ids = list()
for row in rows:
    prod_ids.append(row.product_id)
print(f"{Counter(prod_ids).most_common()}")

[(UUID('bde5d696-1851-4a00-be77-5597abcd7432'), 12), (UUID('560669f9-c891-4a65-ab98-aec368c2ce84'), 11), (UUID('edfaf2e7-63e1-4245-89f0-1d84bc0ee145'), 10), (UUID('f398f466-e1ec-4c2d-ad4c-cea1d53b9d0c'), 10), (UUID('ce72bbaf-ed3d-4ce3-8f52-05933dea0045'), 9), (UUID('37ab86ed-9955-4646-940a-94ba61de464a'), 8), (UUID('342ccd0b-08a5-47bb-b148-be02d5479ea2'), 8), (UUID('9d065c03-57f6-4f4f-9d82-7d7bd06c6e8a'), 8), (UUID('1031cf08-42f5-4840-ae17-ae7a3d66de31'), 7), (UUID('ed4ba001-8727-4912-9d20-3244628021a0'), 7), (UUID('8ca21147-8ef8-44aa-9b75-7fc0a95bb1b2'), 7), (UUID('bb9c0376-d716-4cbf-80b2-ed494fdba82b'), 7), (UUID('eef8150e-556b-494c-858b-34fe6899323a'), 7), (UUID('b3e20836-0daa-4fed-ab4e-bb3b844640db'), 7), (UUID('939f7573-5124-4de0-888f-676043f5543a'), 7), (UUID('12a6837e-3f37-4605-9942-97897e95151f'), 6), (UUID('7a617e97-35ae-4cab-b669-ea66247859e0'), 6), (UUID('aeb3e8ed-a49d-4611-ad75-0d2c6b1fb796'), 6), (UUID('b4671ac4-bf9b-491c-946d-fd5bc03cf88f'), 6), (UUID('d8e9fb90-973f-4080-

In [29]:
#Consulta: quantidade de reclamações por status
query = "SELECT status FROM complaint;"
rows = session.execute(query)
status_list = list()
for row in rows:
    status_list.append(row.status)
print(f"{Counter(status_list).most_common()}")

[('Fechado', 210), ('Em andamento', 202), ('Aberto', 188)]


In [30]:
#Consulta: vendedores com mais reclamações
query = "SELECT seller_id FROM complaint;"
rows = session.execute(query)
seller_list = list()
for row in rows:
    seller_list.append(row.seller_id)
print(f"{Counter(seller_list).most_common()}")

[(UUID('5dbdd57c-2824-4084-b61f-d019eed8c86e'), 18), (UUID('d2ab554c-d33d-402b-a84d-5e432183e9eb'), 17), (UUID('ca1c5d06-0c2e-4ae3-a448-49171d8fb8e0'), 17), (UUID('18f6f570-8e16-4bf5-b370-ec3db6216487'), 14), (UUID('41a8a908-2bff-43f6-b269-fc9e09eb0a21'), 14), (UUID('6a15fd61-186e-4877-8cd4-c5339de7b3f6'), 12), (UUID('c8572a90-a95b-4408-bc65-a0ac8379f43e'), 12), (UUID('aad7e7b2-9cc9-4339-9610-e7496898c309'), 12), (UUID('c91a3291-2ba5-4b69-9352-714818f4c1df'), 12), (UUID('d4f95b18-2f5b-4f67-951b-095166266b60'), 11), (UUID('53e7dfad-6235-454f-99bf-0b7d432ef6c7'), 11), (UUID('7a806e21-18fc-45b9-bb44-7068bec9c983'), 10), (UUID('6ad0bcdf-912d-4df2-afa0-3b79e16d6e60'), 10), (UUID('8eb23e21-682c-4942-91e7-32ee72ae2ac3'), 10), (UUID('495548df-4538-4cf1-8d30-97449c99520b'), 10), (UUID('d3a08b52-e40f-44b1-9121-5bbee5b215e0'), 10), (UUID('fb6f2dd1-b52d-45ed-92eb-acb1a27de891'), 9), (UUID('a77b12ac-e1fe-4745-b13f-7c8bde5a15b7'), 9), (UUID('3666611a-9c34-45f6-b789-5accf6c65286'), 9), (UUID('bd5d489

In [19]:
#Consulta: reclamações separadas por tipo
query = "SELECT type FROM complaint;"
rows = session.execute(query)
type_list = list()
for row in rows:
    type_list.append(row.type)
print(f"{Counter(type_list).most_common()}")

[('Elogio', 80), ('Dúvida', 76), ('Sugestão', 73), ('Reclamação', 71)]


In [25]:
#Consulta: reclamações criadas nos ultimos 30 dias
query = "SELECT ticket_id, creation_date FROM complaint;"
rows = session.execute(query)
date_list = list()
endDateTime = datetime.utcnow()
startDateTime = endDateTime - timedelta(days=30)
for row in rows:
    if row.creation_date >= startDateTime and row.creation_date <= endDateTime:
        date_list.append((row.ticket_id, row.creation_date))
print(f"{date_list}")

[(UUID('410e1619-305e-4fe4-a24f-545bc04d8b73'), datetime.datetime(2023, 7, 20, 13, 25, 50, 808000)), (UUID('8f5f9d10-57b7-4e52-ba02-4af7fdc9c39a'), datetime.datetime(2023, 7, 7, 13, 25, 50, 806000)), (UUID('c40b4ad5-76e5-4ed2-b590-21be64f9e5df'), datetime.datetime(2023, 6, 22, 13, 25, 50, 808000)), (UUID('d3d84153-9b29-4896-a825-9149b673aba4'), datetime.datetime(2023, 7, 14, 13, 25, 50, 809000)), (UUID('ca177fac-69cc-4194-9ec3-cf4d74eac130'), datetime.datetime(2023, 7, 11, 13, 25, 50, 802000)), (UUID('f4e1f220-4fa1-4cc6-bb2f-1d91fb7e6d1b'), datetime.datetime(2023, 6, 28, 13, 25, 50, 807000)), (UUID('d3e7ffe9-3d9e-41cf-ade7-f0a848d46ce1'), datetime.datetime(2023, 7, 11, 13, 25, 50, 802000)), (UUID('197acb9c-66f0-4b46-8a00-060c77d66509'), datetime.datetime(2023, 6, 28, 13, 25, 50, 803000)), (UUID('06683260-55e5-4174-9c78-ae502827189b'), datetime.datetime(2023, 7, 21, 13, 25, 50, 804000)), (UUID('f074a572-a40b-453a-bb82-4e83d1ada6cb'), datetime.datetime(2023, 7, 8, 13, 25, 50, 810000)), (

In [ ]:
cluster.shutdown()